# 오리지널 및 업그레이드 마법공식

# Economic Value
## = Market Cap + Debt + Cash

# Earning Before Interest, Tax, Depreciation, Armotization

# Basic Setting

### Import Modules

In [59]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request as rq
from tqdm import tqdm

In [65]:
INDEX_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'


In [4]:
sample_code = '005930'

## Get EV/EBITDA

In [13]:
index_html = rq.urlopen(INDEX_URL.format(sample_code)).read()
index_soup = BeautifulSoup(index_html, 'html.parser')

In [20]:
ev_cells = index_soup.find('tr',{'id':'p_grid1_14'}).find_all('td')

In [27]:
ev = float(ev_cells[4].string)

## Get Net Income

In [34]:
fs_html = rq.urlopen(FS_URL.format(sample_code)).read()
fs_soup = BeautifulSoup(fs_html, 'html.parser')

In [41]:
is_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class':'rwf'})

In [48]:
ni = float(is_cells[-3].find_all('td')[3].string.replace(',',''))

## Get Asset

In [51]:
bs_cells = fs_soup.find('div', {'id':'divDaechaY'}).find_all('tr', {'class':'rwf'})

In [55]:
asset = float(bs_cells[0].find_all('td')[3].string.replace(',',''))

## Get All Stock code

In [56]:
company = pd.read_csv('company.csv')

In [57]:
code_list = company['종목코드'].dropna()

In [58]:
code_list

0       000155
1       00088K
2       010955
3       051915
4       071055
         ...  
2017    013030
2018    019550
2019    019590
2020    019570
2021    006920
Name: 종목코드, Length: 2021, dtype: object

## Create Crawling Function

In [66]:
def crawling(li):
    result = {}
    
    for code in tqdm(li):
        try:
            # set URL
            INDEX_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
            FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

            # get Company name
            name = company[company['종목코드'] == code]['회사명'].values[0]

            # get EV/EBITDA
            index_html = rq.urlopen(INDEX_URL.format(code)).read()
            index_soup = BeautifulSoup(index_html, 'html.parser')

            ev_cells = index_soup.find('tr',{'id':'p_grid1_14'}).find_all('td')
            ev = float(ev_cells[4].string)

            # get Net Income
            fs_html = rq.urlopen(FS_URL.format(code)).read()
            fs_soup = BeautifulSoup(fs_html, 'html.parser')

            is_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class':'rwf'})
            ni = float(is_cells[-3].find_all('td')[3].string.replace(',',''))

            # get Asset
            bs_cells = fs_soup.find('div', {'id':'divDaechaY'}).find_all('tr', {'class':'rwf'})
            asset = float(bs_cells[0].find_all('td')[3].string.replace(',',''))

            result[name] = [code, ev, ni, asset]
        
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
        
    # convert to DataFrame
    result = pd.DataFrame(result)
    
    # transposing
    result = result.transpose()
    
    # set Column names
    column_names = ['Code', 'EV/EBITDA', 'NI', 'Asset']
    result.columns = column_names
    
    return result

In [67]:
result_df = crawling(code_list)

100%|██████████| 2021/2021 [07:43<00:00,  4.36it/s]


In [69]:
copy_df = result_df.copy()

In [70]:
copy_df.head()

,Code,EV/EBITDA,NI,Asset
하이골드8호,159650,26.39,-34.0,346.0
삼원강재,023000,7.86,88.0,2707.0
LG이노텍,011070,4.7,8883.0,77414.0
S&TC,100840,8.9,125.0,3321.0
동북아13호선박투자,083380,16.67,5.0,81.0


## Screening

In [72]:
copy_df['ROA'] = copy_df.NI / copy_df.Asset

In [75]:
copy_df = copy_df[['Code', 'EV/EBITDA', 'ROA']]

In [79]:
copy_df

,Code,EV/EBITDA,ROA
하이골드8호,159650,26.39,-0.098266
삼원강재,023000,7.86,0.032508
LG이노텍,011070,4.7,0.114747
S&TC,100840,8.9,0.037639
동북아13호선박투자,083380,16.67,0.061728
S&T홀딩스,036530,6.79,0.063989
S&T모티브,064960,5.69,0.069664
한화갤러리아타임월드,027390,4.11,-0.017791
한솔로지스틱스,009180,2.68,0.088147
동성화학,005190,2.99,0.14992


In [ ]:
# EV/EBITDA
# 30% - 3
# 40% - 2
# 30% - 1

# ROA
# 30% - 3
# 40% - 2
# 30% - 1

In [76]:
top_range = int(len(copy_df) * 0.3)
middle_range = int(len(copy_df) * 0.7)

In [86]:
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[:top_range].Code), 'EV/EBITDA_Score'] = 3
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[top_range:middle_range].Code), 'EV/EBITDA_Score'] = 2
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[middle_range:].Code), 'EV/EBITDA_Score'] = 1

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [87]:
copy_df

,Code,EV/EBITDA,ROA,EV/EBITDA_Score
하이골드8호,159650,26.39,-0.098266,1.0
삼원강재,023000,7.86,0.032508,2.0
LG이노텍,011070,4.7,0.114747,3.0
S&TC,100840,8.9,0.037639,2.0
동북아13호선박투자,083380,16.67,0.061728,1.0
S&T홀딩스,036530,6.79,0.063989,2.0
S&T모티브,064960,5.69,0.069664,2.0
한화갤러리아타임월드,027390,4.11,-0.017791,3.0
한솔로지스틱스,009180,2.68,0.088147,3.0
동성화학,005190,2.99,0.14992,3.0


In [88]:
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='ROA', ascending=False)[:top_range].Code), 'ROA_Score'] = 3
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='ROA', ascending=False)[top_range:middle_range].Code), 'ROA_Score'] = 2
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='ROA', ascending=False)[middle_range:].Code), 'ROA_Score'] = 1

In [91]:
copy_df['Total_Score'] = copy_df['EV/EBITDA_Score'] + copy_df['ROA_Score']

In [92]:
copy_df

,Code,EV/EBITDA,ROA,EV/EBITDA_Score,ROA_Score,Total_Score
하이골드8호,159650,26.39,-0.098266,1.0,1.0,2.0
삼원강재,023000,7.86,0.032508,2.0,1.0,3.0
LG이노텍,011070,4.7,0.114747,3.0,3.0,6.0
S&TC,100840,8.9,0.037639,2.0,2.0,4.0
동북아13호선박투자,083380,16.67,0.061728,1.0,2.0,3.0
S&T홀딩스,036530,6.79,0.063989,2.0,2.0,4.0
S&T모티브,064960,5.69,0.069664,2.0,2.0,4.0
한화갤러리아타임월드,027390,4.11,-0.017791,3.0,1.0,4.0
한솔로지스틱스,009180,2.68,0.088147,3.0,3.0,6.0
동성화학,005190,2.99,0.14992,3.0,3.0,6.0


In [95]:
copy_df = copy_df.sort_values(by='Total_Score', ascending=False)

In [98]:
final_result = copy_df.head(20)

In [101]:
final_result = final_result[['Code', 'Total_Score']]

In [102]:
final_result

,Code,Total_Score
동성화학,005190,6.0
LG이노텍,011070,6.0
한솔로지스틱스,009180,6.0
완리,900180,5.0
삼성전자,005930,5.0
삼성전기,009150,5.0
한화갤러리아타임월드,027390,4.0
아트라스BX,023890,4.0
S&T모티브,064960,4.0
S&T홀딩스,036530,4.0
